# Препроцессинг

In [ ]:
import pandas as pd # для удобной работы с датасетом
import re # регулярные выражения

import warnings # для обработки предупреждений
warnings.simplefilter(action='ignore', category=FutureWarning) # игнорируем FutureWarning

Датасетом был выбран был выбран [Flight Price Prediction](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction/), содержащий данные о авиабилетах.

Цель — определить, цену билета по совокупности характеристик.

**Столбцы:**
1) Airline: авиакомпания, осуществляющая перелёт.
2) Flight: номер самолёта.
3) Source City: город отправления.
4) Departure Time: время вылета в общем виде (например: Morning или Early_Morning).
5) Stops: количество промежуточных посадок.
6) Arrival Time: время прибытия, аналогично времени вылета.
7) Destination City Город прибытия.
8) Class: класс битела (бизнес или эконом).
9) Duration: продолительность полёта в часах.
10) Days Left: количество дней между датой покупки билета и датой вылета.
11) Price: цена билета.


In [ ]:
dataset = pd.read_csv("Clean_Dataset.csv",  sep=',', encoding='utf8', index_col=0) 
dataset

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...
300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 27.5+ MB


Меняем категориальную колонку с классом билета на булев признак бизнесс-класса

In [21]:
business_class = [value == 'Business' for value in dataset['class']]
dataset['is_business'] = business_class
dataset.drop(columns=["class"], inplace=True)

dataset.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,duration,days_left,price,is_business
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,2.17,1,5953,False
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,2.33,1,5953,False
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,2.17,1,5956,False
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,2.25,1,5955,False
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,2.33,1,5955,False


Поделим данные примерно пополам - будем передавать в приложение 150 000 записей

In [ ]:
dataset = dataset.sample(frac=1, random_state=42)
dataset.reset_index(drop=True, inplace=True)

In [ ]:
dataset = dataset[:150000]
dataset

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,duration,days_left,price,is_business
0,Air_India,AI-506,Delhi,Morning,one,Early_Morning,Kolkata,19.75,40,7366,False
1,Vistara,UK-706,Kolkata,Morning,one,Night,Mumbai,9.83,42,64831,True
2,Vistara,UK-772,Kolkata,Morning,one,Night,Bangalore,10.50,41,6195,False
3,Vistara,UK-824,Chennai,Night,one,Morning,Delhi,14.50,14,60160,True
4,Air_India,AI-501,Bangalore,Afternoon,one,Night,Mumbai,8.25,20,6578,False
...,...,...,...,...,...,...,...,...,...,...,...
149995,Air_India,AI-809,Mumbai,Morning,one,Night,Bangalore,12.25,10,11469,False
149996,Air_India,AI-676,Kolkata,Morning,one,Night,Hyderabad,11.92,39,5456,False
149997,Air_India,AI-9505,Bangalore,Morning,one,Evening,Chennai,9.92,49,5704,False
149998,Air_India,AI-559,Hyderabad,Early_Morning,one,Evening,Kolkata,35.08,31,7047,False


In [ ]:
dataset.to_csv("data_simple.csv", sep=",", index=False)